In [1]:
import pymc3 as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from theano import shared,tensor
import time 
from datetime import datetime
from sklearn.metrics import mean_squared_error
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
df=pd.read_csv('/kaggle/input/historyapr/EPA_OD_202304.csv')
truth=pd.read_csv('/kaggle/input/history-may/EPA_OD_202305.csv')
df.head()

,SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,...,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId
0,淡水,新北市,32.0,NaN,良好,0.9,0.41,0.3,18.3,35.1,...,11.1,12.1,0.9,NaN,NaN,2023-04-01 00:00:00,0.0,121.449239,25.164500,10
1,新店,新北市,34.0,NaN,良好,0.5,0.38,0.3,24.7,35.9,...,18.4,19.6,1.2,0.8,170,2023-04-01 00:00:00,0.0,121.537778,24.977222,4
2,觀音,桃園市,47.0,NaN,良好,1.2,0.19,0.2,47.5,51.7,...,3.8,4.2,0.3,2.4,78,2023-04-01 00:00:00,0.0,121.082831,25.035567,19
3,新莊,新北市,40.0,NaN,良好,0.7,0.45,0.4,26.8,33.6,...,20.9,23.0,2.0,0.2,84,2023-04-01 00:00:00,0.0,121.432500,25.037972,7
4,萬里,新北市,34.0,NaN,良好,NaN,0.20,0.2,30.2,37.1,...,3.8,4.8,1.0,1.4,217,2023-04-01 00:00:00,0.0,121.689881,25.179667,3


In [14]:
features=['PM10','PM2.5_AVG','PM2.5','SiteId']
for feature in features:
    print(f"{feature} has nan value: ",df[feature].isna().any())

PM10 has nan value:  True
PM2.5_AVG has nan value:  True
PM2.5 has nan value:  True
SiteId has nan value:  False


In [15]:
def preprocessing(data):
    data=data.fillna(data.mean())
    return data
df=preprocessing(df)

/tmp/ipykernel_35/3702725128.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data=data.fillna(data.mean())


In [7]:
# for feature in features[:-2]:
#     df[feature] =( df[feature] - df[feature].mean() ) / df[feature].std()

In [16]:
for feature in features:
    print(f"{feature} has nan value: ",df[feature].isna().any())

PM10 has nan value:  False
PM2.5_AVG has nan value:  False
PM2.5 has nan value:  False
SiteId has nan value:  False


In [17]:
Site=list(df.SiteId.unique())
Site.sort()

In [10]:
SiteTrace={}
number=1
for idx,s in enumerate(Site):
    with pm.Model() as model:
        X0=pm.Data('X0',df[df.SiteId==s][features[0]])
        X1=pm.Data('X1',df[df.SiteId==s][features[1]])
#         X2=pm.Data('X2',df[df.SiteId==s][features[2]])
        y=pm.Data('y',df[df.SiteId==s][features[2]])
        alpha=pm.Normal('alpha',mu=0,sd=10)
        beta=pm.Normal('beta',mu=0,sd=10,shape=2)
        esp=pm.HalfNormal('esp',sd=10)
        mu=pm.Deterministic('mu',alpha+beta[0]*X0+beta[1]*X1)
        y_pred=pm.Normal('y_pred',mu=mu,sd=esp,observed=y)
        trace=pm.sample()
        ppc=pm.sample_posterior_predictive(trace)
        SiteTrace[s]=trace
        if (idx+1)%10==0 or idx==len(Site)-1:
            pklname="dict"+str(number)+".pkl"
            with open(pklname, 'wb') as file:
                pickle.dump(SiteTrace, file)
            SiteTrace={}
            number+=1
#         SiteTrace[s]={'alpha':trace['alpha'].mean(),'beta0':trace['beta'][0].mean(),'beta1':trace['beta'][1].mean()}

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


...

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [13]:
ref={}
for i in range(1,9):
    filename="/kaggle/working/dict"+str(i)+".pkl"
    with open(filename,"rb") as file:
        ref.update(pickle.load(file))
ref

{1: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 2: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 3: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 4: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 5: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 6: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 7: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 8: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 9: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 10: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 11: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 12: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 13: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 14: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 15: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 16: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 17: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 18: <

In [18]:
def calculate_result(row):
    site=int(row['SiteId'])
    a=ref[site]['alpha'].mean()
    b0=ref[site]['beta'][0].mean()
    b1=ref[site]['beta'][1].mean()
    b2=ref[site]['beta'][2].mean()
    result = a + b0 * row['AQI'] + b1 * row['PM10'] + b2 * row['PM2.5_AVG']
    return result
df['result'] = df.apply(calculate_result, axis=1)

In [19]:
mse = mean_squared_error(df['PM2.5'].values, df['result'].values)
print("training mse:",mse)

training mse: 1091.1900533941139


In [20]:
# for feature in features[:-2]:
#     truth[feature] =( truth[feature] - truth[feature].mean() ) / truth[feature].std()
# truth.fillna(value=truth['PM2.5'].mean(),inplace=True)
truth=preprocessing(truth)
truth['result'] = truth.apply(calculate_result, axis=1)
mse = mean_squared_error(truth['PM2.5'].values, truth['result'].values)
print("testing mse:",mse)

/tmp/ipykernel_35/3702725128.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data=data.fillna(data.mean())


testing mse: 701.410764226237
